<h1><center><b>REINFORCEMENT LEARNING</b></center></h1>
<h1><center><b>Final Project - Part 1.2</b></center></h1>



<h3><center>Lunar Lander v2</center></h3>

---


<h3><center>DDPG</center></h3>

Names:  
<font color='red'>
Zuriya Ansbacher ID. 208532515  
Akiva Bruno Melka ID. 332629393  
</font>

https://github.com/zuriyaAnsbacher/Reinforcement_Project

# **1. Install Environment**

In [ ]:
!sudo apt-get update
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

# **2. Import Libraries**

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
from gym.wrappers.monitoring import video_recorder

import os
import copy
import random
import argparse
import numpy as np
from statistics import mean
from collections import deque
from dataclasses import dataclass
from itertools import product

import torch
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import torch.optim as optim
from torch.optim import Adam

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [ ]:
!pip install gym[box2d]

# **3. Utility functions**

In [ ]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video(file):
  mp4list = glob.glob(file)
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

""" Utils Functions """

def set_seed(env, seed):
    np.random.seed(seed)
    env.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

def initialize_layer(layer, w):
    layer.weight.data.uniform_(-w, w)
    layer.bias.data.uniform_(-w, w)


def test(agent, env, paths, model_name, episode_num=100, steps_num=1000, record=False, record_freq=50):
  print("Start evaluation...")

  agent.set_eval()
  scores_list = []
  success = False

  for episode in range(1, episode_num + 1):
      env = wrap_env(env)  #todo: check
      state = env.reset()
      score = 0

      for step in range(1, steps_num + 1):

        action = agent.get_action(state, just_greedy=True)
        next_state, reward, done, _ = env.step(action)
        score += reward
        state = next_state
        if done: 
          break

      scores_list.append(score)
      print(f'Episode: {episode}, Test score: {score:.02f}')

  if mean(scores_list) >= 200:
    success = True
    plt.axhline(200, c='gray', label='Goal', alpha=0.7)
    plt.plot(scores_list, 'c', label='Score', alpha=0.7)
    plt.legend(loc='lower right')
    plt.xlabel('Episodes')
    plt.ylabel('Scores')
    plt.title(f'{model_name} Test. Average score: {mean(scores_list):.02f}')
    plt.show()

    plt.close()

  else:
    print(f'The evaluation has not achieved the goal yet. The model will be training more..')
    # env.monitor.close()  #todo: check 

  return success

In [ ]:
@ dataclass
class Sample:
    state: np.ndarray
    action: int or np.ndarray
    reward: float
    next_state: np.ndarray
    done: bool


class SamplesMemory:
    def __init__(self, max_size, device):
        self.max_size = max_size
        self.device = device
        self.memory_buffer = deque(maxlen=max_size)  # maxlen ensure that samples num won't exceed

    def add_sample(self, state, action, reward, next_state, done):
        sample = Sample(state, action, reward, next_state, done)
        self.memory_buffer.append(sample)

    def get_batch(self, batch_size, continuous_action=False):
        batch = random.sample(self.memory_buffer, batch_size)
        f = lambda x, my_type: torch.tensor(np.vstack(x), device=self.device, dtype=my_type)

        state_batch = f([sample.state for sample in batch], torch.float)
        action_batch = f([sample.action for sample in batch], torch.float) if continuous_action \
            else f([sample.action for sample in batch], torch.long)
        reward_batch = f([sample.reward for sample in batch], torch.float)
        next_state_batch = f([sample.next_state for sample in batch], torch.float)
        done_batch = f([sample.done for sample in batch], torch.float)

        return state_batch, action_batch, reward_batch, next_state_batch, done_batch

In [ ]:
""" Ornstein-Uhlenbeck Noise """

class OU_Noise:
    def __init__(self, action_size, mu=0., theta=0.15, sigma=0.2):
        self.action_size = action_size
        self.mu = np.ones(self.action_size) * mu
        self.theta = theta
        self.sigma = sigma
        self.state = None
        self.reset()

    def reset(self):
        self.state = copy.copy(self.mu)

    def add_noise(self):
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(len(x))
        self.state = x + dx
        return self.state

# **4. Models**

In [ ]:
""" Models """
class Actor(nn.Module):
    def __init__(self, input_size, output_size, hidd1_size, hidd2_size, weight_init=3e-3):
        super(Actor, self).__init__()

        self.fc1 = nn.Sequential(nn.Linear(input_size, hidd1_size), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(hidd1_size, hidd2_size), nn.ReLU())
        self.fc3 = nn.Linear(hidd2_size, output_size)

        self.weight_init = weight_init
        initialize_layer(self.fc3, self.weight_init)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = torch.tanh(self.fc3(x))
        return x

class Critic(nn.Module):
    def __init__(self, input_size, action_size, hidd1_size, hidd2_size, weight_init=3e-3):
        super(Critic, self).__init__()

        self.fc1 = nn.Sequential(nn.Linear(input_size, hidd1_size), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(hidd1_size + action_size, hidd2_size), nn.ReLU())
        self.fc3 = nn.Linear(hidd2_size, 1)

        self.weight_init = weight_init
        initialize_layer(self.fc3, self.weight_init)

    def forward(self, state, action):
        x = self.fc1(state)
        x = torch.cat((x, action), dim=1)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


# **5. Agent**

In [ ]:
""" Agent """

class DDPGAgent:
    def __init__(self, state_size, action_size, batch_size, gamma, lr_actor, lr_critic, 
                 tau, max_eps, min_eps, eps_decay, hidden_actor, hidden_critic, 
                 memory_size, device):

        self.model_name = 'DDPG'

        self.state_size = state_size
        self.action_size = action_size

        self.gamma = gamma
        self.tau = tau
        self.cur_eps = max_eps
        self.min_eps = min_eps
        self.eps_decay = eps_decay

        self.device = device

        self.memory = SamplesMemory(memory_size, device)
        self.batch_size = batch_size
        self.noise = OU_Noise(self.action_size)

        self.hidd1_actor = hidden_actor[0]
        self.hidd2_actor = hidden_actor[1]
        self.actor = Actor(self.state_size, self.action_size, self.hidd1_actor, self.hidd2_actor).to(device)
        self.actor_target = Actor(self.state_size, self.action_size, self.hidd1_actor, self.hidd2_actor).to(device)
        self.actor_optimizer = Adam(self.actor.parameters(), lr=lr_actor)

        self.hidd1_critic = hidden_critic[0]
        self.hidd2_critic = hidden_critic[1]
        self.critic = Critic(self.state_size, self.action_size, self.hidd1_critic, self.hidd2_critic).to(device)
        self.critic_target = Critic(self.state_size, self.action_size, self.hidd1_critic, self.hidd2_critic).to(device)
        self.critic_optimizer = Adam(self.critic.parameters(), lr=lr_critic)
        self.copy_params_to_target()

        self.critic_loss_func = nn.MSELoss()

    def get_action(self, state, just_greedy=False):
        state = torch.tensor(state, dtype=torch.float, device=self.device)
        self.actor.eval()
        with torch.no_grad():
            action = self.actor(state).cpu().data.numpy()
        self.actor.train()
        action += self.cur_eps * self.noise.add_noise()
        action = np.clip(action, -1, 1)

        return action

    def decrement_epsilon(self):
        return max(self.cur_eps - self.eps_decay, self.min_eps)

    def save(self, file_path):
        torch.save(self.actor.state_dict(), file_path)

    def load(self, file_path):
        self.actor.load_state_dict(torch.load(file_path, map_location=self.device))

    def set_eval(self):
        self.actor.eval()

    def copy_params_to_target(self, curr_tau=1.0):
        for net, target_net in zip([self.actor, self.critic], [self.actor_target, self.critic_target]):
            for params, target_params in zip(net.parameters(), target_net.parameters()):
                target_params.data.copy_(params.data * curr_tau + target_params.data * (1.0 - curr_tau))

    def learn(self):
        states, actions, rewards, next_states, dones = self.memory.get_batch(self.batch_size, continuous_action=True)

        # q values
        next_action = self.actor_target(next_states)
        q_val = self.critic(states, actions)
        next_q = self.critic_target(next_states, next_action)

        # formula (bellman equation)
        q_prime = rewards + (self.gamma * next_q * (1 - dones))

        # critic: loss and optimizer
        critic_loss = self.critic_loss_func(q_val, q_prime)
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        nn.utils.clip_grad_norm_(self.critic.parameters(), 1)
        self.critic_optimizer.step()

        # actor: loss and optimizer
        actions_pred = self.actor(states)
        actor_loss = -self.critic(states, actions_pred).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        self.copy_params_to_target(curr_tau=self.tau)

        self.cur_eps = self.decrement_epsilon()
        self.noise.reset()

    def train(self, env, paths, episodes_num, steps_num, learn_freq, update_factor, 
              record=False, record_freq=50):
        scores_list = []
        count_test = 1

        for episode in range(1, episodes_num + 1):
            print(f'Episode: {episode} | Steps: ', end='')
            state = env.reset()
            self.noise.reset()

            score = 0
            for step in range(1, steps_num + 1):
                if step % 100 == 0:
                    print(step, end=' ')
                action = self.get_action(state)
                next_state, reward, done, _ = env.step(action)

                self.memory.add_sample(state, action, reward, next_state, done)
                if len(self.memory.memory_buffer) > self.batch_size and step % learn_freq == 0:
                    for _ in range(update_factor):
                        self.learn()

                score += reward
                state = next_state

                if done:
                    break

            print(f'| Score: {score}')
            scores_list.append(score)

            if len(scores_list) >= 50 and mean(scores_list[-20:]) >= 200 and episode >= count_test + 5:  # last condition: for not doing test every episode, when the learning starts to be better
              count_test = episode
              # create_plot(scores_list, self.model_name)
              success = test(self, env, paths, self.model_name)
              if success:
                print(f'Mission accomplished in episode {episode}!')
                show_video()  # check
                break

        env.close()

# **6. Main**

In [ ]:
def main():
  parser = argparse.ArgumentParser()
  parser.add_argument('--model', default='ddpg')
  parser.add_argument('--train', default=True, help='train agent')

  parser.add_argument('--memory_size', type=int, default=100000, help='memory buffer size')
  parser.add_argument('--episodes', type=int, default=800, help='Number of episodes used in training')
  parser.add_argument('--max_eps', default=1.0)
  parser.add_argument('--min_eps', default=0.01)
  

  parser.add_argument('--max_steps', default=800)
  parser.add_argument('--batch_size', default=256)
  parser.add_argument('--gamma', default=0.9849902858712885)
  parser.add_argument('--tau', default=0.008711453313535608)
  parser.add_argument('--lr_actor', default=0.0021267736280409758)
  parser.add_argument('--lr_critic', default=0.0032480168831600692)
  parser.add_argument('--learn_freq', default=8)
  parser.add_argument('--update_factor', default=10)
  parser.add_argument('--eps_decay', default=0.0008006776266587679)
  parser.add_argument('--hidden_actor', default=[256, 128])
  parser.add_argument('--hidden_critic', default=[128, 128])

  args = parser.parse_args(args=[])

  paths = None

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(device)

  env = gym.make('LunarLanderContinuous-v2')
  env.reset()

  state_size = env.observation_space.shape[0] 
  action_size = env.action_space.shape[0]

  agent_params = [state_size, action_size, args.batch_size, args.gamma, args.lr_actor,
                    args.lr_critic, args.tau, args.max_eps, args.min_eps, args.eps_decay,
                    args.hidden_actor, args.hidden_critic, args.memory_size, device]

  agent = DDPGAgent(*agent_params)

  if args.train:
    train_params = [env, paths, args.episodes, args.max_steps, args.learn_freq, args.update_factor]
    agent.train(*train_params)


if __name__ == '__main__':
  main()